In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sid import get_simulate_func
from sid.time import get_date

from src.config import BLD
from src.contact_models.get_contact_models import (
    get_all_contact_models,
    get_christmas_contact_models,
)
from src.create_initial_states.create_initial_conditions import (
    create_initial_conditions,
)

# Inputs that won't change

In [2]:
params = pd.read_pickle(BLD / "start_params.pkl")
params.to_csv(BLD / "start_params.csv")

share_known_cases = pd.read_pickle(
    BLD / "data" / "processed_time_series" / "share_known_cases.pkl"
)

# Inputs that will change

In [3]:
# replace this with the full initial states later on
df = pd.read_parquet(BLD / "data" / "debug_initial_states.parquet")

# these will change depending on the scenario in which we are
contact_models = get_all_contact_models("full", 2)
policies = None

# This is only until we have estimated infection probabilities!

In [4]:
def _build_infection_probs(names):
    index_tuples = [("infection_prob", mod_name, mod_name) for mod_name in names]
    df = pd.DataFrame(index=pd.MultiIndex.from_tuples(index_tuples))
    df.index.names = ["category", "subcategory", "name"]
    df = df.reset_index()
    prob_dict = {
        "educ": 0.02,
        "work": 0.1,
        "household": 0.2,
        "other": 0.1,
        "christmas": 0.2,
        "holiday_preparation": 0.1,
    }
    full_prob_dict = {}
    for mod_name in names:
        for k, v in prob_dict.items():
            if k in mod_name:
                full_prob_dict[mod_name] = v
        assert (
            mod_name in full_prob_dict
        ), f"No infection probability for {mod_name} specified."

    df["value"] = df["name"].map(full_prob_dict.get)
    df = df.set_index(["category", "subcategory", "name"])
    return df

In [5]:
# set the infection probabilities
infection_probs = _build_infection_probs(contact_models.keys())

params = pd.concat([infection_probs, params])
params.head()

value
category       subcategory      name                   
infection_prob households       households         0.20
               educ_school_0    educ_school_0      0.02
               educ_school_1    educ_school_1      0.02
               educ_school_2    educ_school_2      0.02
               educ_preschool_0 educ_preschool_0   0.02

In [7]:
params.loc["infection_prob"].index.get_level_values("subcategory")

Index(['households', 'educ_school_0', 'educ_school_1', 'educ_school_2',
       'educ_preschool_0', 'educ_nursery_0', 'work_non_recurrent',
       'work_recurrent_daily', 'work_recurrent_weekly_0',
       'work_recurrent_weekly_1', 'work_recurrent_weekly_2',
       'work_recurrent_weekly_3', 'work_recurrent_weekly_4',
       'work_recurrent_weekly_5', 'work_recurrent_weekly_6',
       'work_recurrent_weekly_7', 'work_recurrent_weekly_8',
       'work_recurrent_weekly_9', 'work_recurrent_weekly_10',
       'work_recurrent_weekly_11', 'work_recurrent_weekly_12',
       'work_recurrent_weekly_13', 'other_non_recurrent',
       'other_recurrent_daily', 'other_recurrent_weekly_0',
       'other_recurrent_weekly_1', 'other_recurrent_weekly_2',
       'other_recurrent_weekly_3', 'holiday_preparation', 'christmas_full_0',
       'christmas_full_1', 'christmas_full_2'],
      dtype='object', name='subcategory')

In [10]:
# set the degree of responsibility for self-isolation

react_to_christmas_contact_multipliers = [
    ("contact_symptomatic_multiplier", 0.15),
    ("contact_positive_test_multiplier", 0.05),
]

models_with_post_christmas_isolation = [
    # 'households',  <- all affected
    # 'educ_school_0', 'educ_school_1', 'educ_school_2', <- vacation
    # 'educ_preschool_0', 'educ_nursery_0', <- vacation
    "work_non_recurrent",
    "work_recurrent_daily",
    "work_recurrent_weekly_0",
    "work_recurrent_weekly_1",
    "work_recurrent_weekly_2",
    "work_recurrent_weekly_3",
    "work_recurrent_weekly_4",
    "work_recurrent_weekly_5",
    "work_recurrent_weekly_6",
    "work_recurrent_weekly_7",
    "work_recurrent_weekly_8",
    "work_recurrent_weekly_9",
    "work_recurrent_weekly_10",
    "work_recurrent_weekly_11",
    "work_recurrent_weekly_12",
    "work_recurrent_weekly_13",
    "other_non_recurrent",
    "other_recurrent_daily",
    "other_recurrent_weekly_0",
    "other_recurrent_weekly_1",
    "other_recurrent_weekly_2",
    "other_recurrent_weekly_3",
    # 'holiday_preparation', 'christmas_full_0',
    # 'christmas_full_1', 'christmas_full_2'
]

for name, multiplier in react_to_christmas_contact_multipliers:
    for model in models_with_post_christmas_isolation:
        params.loc[(model, name, name), "value"] = multiplier

params.tail()

/home/klara/anaconda3/envs/sid-germany/lib/python3.8/site-packages/IPython/core/interactiveshell.py:2877: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(


,,,value
category,subcategory,name,
other_recurrent_daily,contact_positive_test_multiplier,contact_positive_test_multiplier,0.05
other_recurrent_weekly_0,contact_positive_test_multiplier,contact_positive_test_multiplier,0.05
other_recurrent_weekly_1,contact_positive_test_multiplier,contact_positive_test_multiplier,0.05
other_recurrent_weekly_2,contact_positive_test_multiplier,contact_positive_test_multiplier,0.05
other_recurrent_weekly_3,contact_positive_test_multiplier,contact_positive_test_multiplier,0.05


# Create the initial conditions

In [5]:
simulation_start = pd.Timestamp("2020-12-01")
initial_start = simulation_start - pd.Timedelta(weeks=2)
simulation_end = simulation_start + pd.Timedelta(weeks=4)

initial_conditions = create_initial_conditions(
    start=initial_start,
    end=simulation_start - pd.Timedelta(days=1),
    reporting_delay=7,
    seed=99,
)

# Run the simulation

In [6]:
simulate = get_simulate_func(
    params=params,
    initial_states=df,
    contact_models=contact_models,
    duration={"start": simulation_start, "end": simulation_end},
    contact_policies=policies,
    initial_conditions=initial_conditions,
    events=None,
    seed=384,
)

In [7]:
res = simulate(params)

Start the simulation...
2020-12-29: 100%|██████████| 29/29 [00:23<00:00,  1.22it/s]
